## Step 1. Install dependencies and ETL process 

### Download data

In [1]:
import kagglehub

# Download latest version
path = kagglehub.dataset_download("mlippo/average-global-iq-per-country-with-other-stats")

print("Path to dataset files:", path)

# Dowload additional dataset

path2 = kagglehub.dataset_download("mexwell/global-iq-1950-2050")

print("Path to dataset files:", path2)

/Users/Gala/repo/capstone_project/.venv/lib/python3.9/site-packages/urllib3/__init__.py:35: NotOpenSSLWarning: urllib3 v2 only supports OpenSSL 1.1.1+, currently the 'ssl' module is compiled with 'LibreSSL 2.8.3'. See: https://github.com/urllib3/urllib3/issues/3020
  warnings.warn(
/Users/Gala/repo/capstone_project/.venv/lib/python3.9/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


Path to dataset files: /Users/Gala/.cache/kagglehub/datasets/mlippo/average-global-iq-per-country-with-other-stats/versions/3
Path to dataset files: /Users/Gala/.cache/kagglehub/datasets/mexwell/global-iq-1950-2050/versions/1


Create path to main file

In [2]:
import os
import pandas as pd

# Create path to file 
csv_file_path = os.path.join(path, "avgIQpercountry.csv")

# Read data
df = pd.read_csv(csv_file_path)

# Print information about data
df.info()
df.head()


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 193 entries, 0 to 192
Data columns (total 10 columns):
 #   Column                          Non-Null Count  Dtype  
---  ------                          --------------  -----  
 0   Rank                            193 non-null    int64  
 1   Country                         193 non-null    object 
 2   Average IQ                      193 non-null    float64
 3   Continent                       193 non-null    object 
 4   Literacy Rate                   193 non-null    float64
 5   Nobel Prices                    193 non-null    int64  
 6   HDI (2021)                      179 non-null    float64
 7   Mean years of schooling - 2021  179 non-null    float64
 8    GNI - 2021                     179 non-null    float64
 9   Population - 2023               193 non-null    object 
dtypes: float64(5), int64(2), object(3)
memory usage: 15.2+ KB


,Rank,Country,Average IQ,Continent,Literacy Rate,Nobel Prices,HDI (2021),Mean years of schooling - 2021,GNI - 2021,Population - 2023
0,1,Japan,106.48,Asia,0.99,29,0.925,13.4,42274.0,123294513
1,2,Taiwan,106.47,Asia,0.96,4,NaN,NaN,NaN,10143543
2,3,Singapore,105.89,Asia,0.97,0,0.939,11.9,90919.0,6014723
3,4,Hong Kong,105.37,Asia,0.94,1,0.952,12.2,62607.0,7491609
4,5,China,104.10,Asia,0.96,8,0.768,7.6,17504.0,1425671352


Import packages

In [3]:
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

import plotly.graph_objs as go
import plotly.offline as pyo
import plotly.express as px
import plotly.io as pio
import geopandas as gpd

from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error
from sklearn.preprocessing import StandardScaler
from sklearn.decomposition import PCA
from sklearn.linear_model import LinearRegression, Ridge, Lasso
from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import r2_score
from sklearn.model_selection import cross_val_score
from sklearn.metrics import mean_squared_error
from scipy.stats import pearsonr

import warnings
warnings.filterwarnings("ignore")

# Check missing data
print(df.isnull().sum())

#Check duplicates
df.duplicated().sum()

#Strip spacecs from column names
df.columns = df.columns.str.strip()

# Rename columns
df.rename(columns={'Nobel Prices': 'Nobel Prizes'}, inplace=True)

Rank                               0
Country                            0
Average IQ                         0
Continent                          0
Literacy Rate                      0
Nobel Prices                       0
HDI (2021)                        14
Mean years of schooling - 2021    14
 GNI - 2021                       14
Population - 2023                  0
dtype: int64


Rename columns

In [4]:
AverageIQColName = 'Average IQ'
GNI_2021ColName = 'GNI - 2021'
HDI_2021ColName = 'HDI (2021)'
Literacy_RateColName = 'Literacy Rate'
Schooling_2021ColName = 'Mean years of schooling - 2021'
Nobel_PrizesColName = 'Nobel Prizes'
CountryColName = 'Country'
Population_2023ColName = 'Population - 2023'

### Data Cleaning and Preprocessing

In [5]:
# Filling in the gaps with median values
df[HDI_2021ColName].fillna(df[HDI_2021ColName].median(), inplace=True)
df[Schooling_2021ColName].fillna(df[Schooling_2021ColName].median(), inplace=True)
df[GNI_2021ColName].fillna(df[GNI_2021ColName].median(), inplace=True)

# Replace population column to numeric 
df[Population_2023ColName] = pd.to_numeric(df[Population_2023ColName].replace({',': ''}, regex=True), errors='coerce')
df[Population_2023ColName].fillna(0, inplace=True)  # Handle any NaNs after conversion
df = df.astype({Population_2023ColName: 'int64'})

# Outlier detection and handling
for column in [HDI_2021ColName, Schooling_2021ColName, GNI_2021ColName]:
    Q1 = df[column].quantile(0.25)
    Q3 = df[column].quantile(0.75)
    IQR = Q3 - Q1
    lower_bound = Q1 - 1.5 * IQR
    upper_bound = Q3 + 1.5 * IQR
    df = df[(df[column] >= lower_bound) & (df[column] <= upper_bound)]

In [6]:
print("Print data")

print(df.head(10))

Print data
    Rank      Country  Average IQ Continent  Literacy Rate  Nobel Prizes  \
0      1        Japan      106.48      Asia           0.99            29   
1      2       Taiwan      106.47      Asia           0.96             4   
3      4    Hong Kong      105.37      Asia           0.94             1   
4      5        China      104.10      Asia           0.96             8   
5      6  South Korea      102.35      Asia           0.98             0   
6      7      Belarus      101.60    Europe           1.00             2   
7      8      Finland      101.20    Europe           1.00             5   
9     10      Germany      100.74    Europe           0.99           111   
10    11  Netherlands      100.74    Europe           0.99            22   
11    12      Estonia      100.72    Europe           1.00             0   

    HDI (2021)  Mean years of schooling - 2021  GNI - 2021  Population - 2023  
0        0.925                            13.4     42274.0          1232

In [7]:
df.describe().round(2)

,Rank,Average IQ,Literacy Rate,Nobel Prizes,HDI (2021),Mean years of schooling - 2021,GNI - 2021,Population - 2023
count,187.00,187.00,187.00,187.00,187.00,187.00,187.00,1.870000e+02
mean,99.08,81.57,0.86,5.90,0.72,8.96,17914.90,4.270397e+07
std,55.11,13.20,0.18,32.43,0.14,3.06,16770.24,1.526244e+08
min,1.00,42.99,0.19,0.00,0.38,2.10,732.00,0.000000e+00
25%,52.50,73.90,0.79,0.00,0.61,6.70,4707.00,2.639762e+06
50%,99.00,82.10,0.95,0.00,0.74,9.40,12672.00,1.014354e+07
75%,146.50,91.01,0.99,1.00,0.81,11.35,26244.50,3.400967e+07
max,193.00,106.48,1.00,400.00,0.96,14.10,64765.00,1.428628e+09


In [8]:
df.iloc[:, :].describe().T.sort_values(by='std', ascending=False)\
.style.bar(subset=["max"], color='blue')\
.bar(subset=["mean"], color='green')

,count,mean,std,min,25%,50%,75%,max
Population - 2023,187.000000,42703973.508021,152624361.422261,0.000000,2639762.500000,10143543.000000,34009669.500000,1428627663.000000
GNI - 2021,187.000000,17914.898396,16770.244597,732.000000,4707.000000,12672.000000,26244.500000,64765.000000
Rank,187.000000,99.080214,55.108193,1.000000,52.500000,99.000000,146.500000,193.000000
Nobel Prizes,187.000000,5.898396,32.433975,0.000000,0.000000,0.000000,1.000000,400.000000
Average IQ,187.000000,81.568289,13.195865,42.990000,73.905000,82.100000,91.010000,106.480000
Mean years of schooling - 2021,187.000000,8.956150,3.060782,2.100000,6.700000,9.400000,11.350000,14.100000
Literacy Rate,187.000000,0.860267,0.181731,0.190000,0.790000,0.950000,0.990000,1.000000
HDI (2021),187.000000,0.719155,0.144798,0.385000,0.607000,0.745000,0.814000,0.961000


### Feature engineering

In [9]:
# Create new featude "GDP per capita"
df['GNI per capita'] = df[GNI_2021ColName] / df[Population_2023ColName]


The feature "GDP per capita" is an indicator that measures the economic well-being of a country by dividing the total income by the number of people in the country.

Create path to additional file

In [10]:
df_2002 = pd.read_csv('../data/iq-2002-2.csv', delimiter=",")  # Additional dataset
df_2002.info()
df_2002.head()

# Check missing data
print(df_2002.isnull().sum())

#Check duplicates
df_2002.duplicated().sum()


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 183 entries, 0 to 182
Data columns (total 3 columns):
 #   Column      Non-Null Count  Dtype 
---  ------      --------------  ----- 
 0   Rank        183 non-null    int64 
 1   Country     183 non-null    object
 2   Average IQ  183 non-null    int64 
dtypes: int64(2), object(1)
memory usage: 4.4+ KB
Rank          0
Country       0
Average IQ    0
dtype: int64


np.int64(0)

### Save dataframe

In [11]:
# Save DataFrame to a pickle file
df.to_pickle('data.pkl')
df_2002.to_pickle('data_2002.pkl')